# <h1 align="center"><font color="gree">Mistral AI Agents API Tutorial</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [Obtenha a sua chave de API do Mistral AI](https://console.mistral.ai/api-keys)

* [DataCamp - Mistral AI](https://www.youtube.com/watch?v=VkqYmwIIeOg)

* [Mistral AI Cookbook](https://github.com/mistralai/cookbook/tree/main)

* [Visão geral de modelos](https://docs.mistral.ai/getting-started/models/models_overview/)

# <font color="red">Começando</font>

In [1]:
from mistralai import Mistral
import os

# Alternativamente, você pode usar variáveis de ambiente
MISTRALAI_API_KEY = os.environ.get("MISTRALAI_API_KEY") 

client = Mistral(api_key=MISTRALAI_API_KEY)

ml_agent = client.beta.agents.create(
    model="mistral-medium-2505",
    name="ml-agent",
    description="Asistente de Machine learning",
    instructions="Você é um especialista em Machine learning. Dá conselhos práticos e ações.",
)


In [5]:
print(f"Agent created: {ml_agent.id}")
print("\n")
print(f"Agent name: {ml_agent.name}")
print("\n")
print(f"Agent description: {ml_agent.description}")
print("\n")
print(f"Agent instructions: {ml_agent.instructions}")
print("\n")
print(f"Agent model: {ml_agent.model}")


Agent created: ag_019818e32a1c73d88638d1a73a709ff8


Agent name: ml-agent


Agent description: Asistente de Machine learning


Agent instructions: Você é um especialista em Machine learning. Dá conselhos práticos e ações.


Agent model: mistral-medium-2505


* ``mistral-medium`` é um modelo pago com um equilíbrio entre desempenho e custo
* ``name`` deve ser único para manter conversas separadas
* ``instructions`` é o prompt do sistema que guia o comportamento do agente

Os Agents API usam IDs de agentes para diferenciar um agente de outro


<font color="gree">Iniciando uma conversa</font>

* Para começar a fazer perguntas, use o comando ``start``
* Deve passar o ``ID`` de agente


In [6]:
response = client.beta.conversations.start(
    agent_id=ml_agent.id,
    inputs="""Devo usar Random Forest ou XGBoost para um dataset de 
              5000 amostras? Responda em uma sentença apenas.""",
)

In [7]:
type(response)

mistralai.models.conversationresponse.ConversationResponse

In [ ]:
response.outputs

[MessageOutputEntry(content='Para um dataset de 5000 amostras, XGBoost geralmente oferece melhor desempenho e eficiência.', object='entry', type='message.output', created_at=datetime.datetime(2025, 7, 17, 15, 6, 14, 383662, tzinfo=TzInfo(UTC)), completed_at=datetime.datetime(2025, 7, 17, 15, 6, 14, 708149, tzinfo=TzInfo(UTC)), id='msg_019818ebf7ef72b79a5092bf66a696eb', agent_id='ag_019818e32a1c73d88638d1a73a709ff8', model='mistral-medium-2505', role='assistant')]

* O array de saídas (``outputs``) conterá outros objetos de mensagem como chamadas de ferramentas.

In [11]:
print(response.outputs[0].content)

Para um dataset de 5000 amostras, XGBoost geralmente oferece melhor desempenho e eficiência.


Isto é um workflow básico para criar agentes e iniciar conversas com eles.


# <font color="red">Mergulho profundo na API de agentes Mistral</font>

## <font color="gree">``1.`` Criando agentes eficazes</font>

* O desempenho do agente é determinado por:

1. Escolha do modelo
2. Prompt do sistema
3. Parâmetros de conclusão (`Completion`)

* Escolha o modelo certo na página de [visão geral de modelos](https://docs.mistral.ai/getting-started/models/models_overview/)


In [ ]:
# Agentes especializados com diferentes configurações:
data_agent = client.beta.agents.create(
    model="mistral-medium-latest",
    name="data-expert",
    description="Especialista em pré-processamento de dados",
    instructions="Especialista em limpeza de dados e engenharia de recursos (feature engineering)",
    completion_args={"temperature": 0.1},
)

print(f"Data agent: {data_agent.id}")

Data agent: ag_01981910206d75709f9cf966b4e184f3


In [13]:
print(data_agent)

model='mistral-medium-latest' name='data-expert' id='ag_01981910206d75709f9cf966b4e184f3' version=0 created_at=datetime.datetime(2025, 7, 17, 15, 45, 44, 49338, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2025, 7, 17, 15, 45, 44, 49340, tzinfo=TzInfo(UTC)) instructions='Especialista em limpeza de dados e engenharia de recursos (feature engineering)' tools=[] completion_args=CompletionArgs(stop=None, presence_penalty=None, frequency_penalty=None, temperature=0.0, top_p=None, max_tokens=None, random_seed=None, prediction=None, response_format=None, tool_choice='auto') description='Especialista em pré-processamento de dados' handoffs=None object='agent'


* Use modelos premium (pagos) para cenários de alto impacto que exigem forte ``raciocínio/codificação``. Boas opções são:
    * ``Codestral 2`` para codificação
    * ``Magistral medium`` para raciocínio
* Faça o ``prompt do sistema`` o mais detalhado possível - dedique a maior parte do seu tempo aqui
* A ``temperatura`` controla a aleatoriedade da próxima palavra
    * ``0.1-0.5`` para precisão factual (codificação, documentos técnicos)
    * ``0.5-1`` para criatividade (brainstorming (gerar ideias), menos robótico)

## <font color="gree">``2.``Gerenciando conversas com um agente</font>

* A maioria dos frameworks de construção de agentes não possui memória embutida
* Os casos de uso em produção são diferentes
* O ``Mistral`` vem com memória embutida e gerenciamento de conversas

<font color="red">``I.`` Iniciando uma nova conversa</font>

* O comando ``start`` sempre espera um novo ID de agente

In [14]:
response = client.beta.conversations.start(
    agent_id=ml_agent.id,
    inputs="""Quais são as melhores práticas para codificar variáveis 
              categóricas em aprendizado de máquina? Liste três.""",
)

print(response.outputs[0].content)

Aqui estão três melhores práticas para codificar variáveis categóricas em aprendizado de máquina:

1. **One-Hot Encoding**:
   - **Descrição**: Transforma cada categoria em uma coluna binária (0 ou 1). Cada coluna representa uma categoria única.
   - **Quando usar**: Quando as categorias não têm uma ordem intrínseca (por exemplo, cores, países).
   - **Exemplo**: Se você tem uma variável "Cor" com categorias "Vermelho", "Azul" e "Verde", o One-Hot Encoding criará três colunas binárias, cada uma representando uma dessas cores.

2. **Label Encoding**:
   - **Descrição**: Atribui um número inteiro único a cada categoria.
   - **Quando usar**: Quando as categorias têm uma ordem intrínseca (por exemplo, níveis de educação: "Ensino Fundamental", "Ensino Médio", "Ensino Superior").
   - **Exemplo**: Se você tem uma variável "Nível de Educação" com categorias "Ensino Fundamental", "Ensino Médio" e "Ensino Superior", o Label Encoding pode atribuir 0, 1 e 2, respectivamente.

3. **Embedding**:
 

* Todos os objetos de ``response`` de conversação também têm um ``ID``.

In [15]:
print(response.conversation_id)

conv_01981929efd671908fe477e0bc9cf15c


<font color="red">``II.`` Continuando uma conversa existente</font>

In [16]:
# Continuar uma conversa existente:
follow_up = client.beta.conversations.append(
    conversation_id=response.conversation_id,
    inputs="Qual foi a minha primeira pergunta?",
)

print(follow_up.outputs[0].content)

Sua primeira pergunta foi: "Quais são as melhores práticas para codificar variáveis categóricas em aprendizado de máquina? Liste três."


In [17]:
follow_up = client.beta.conversations.append(
    conversation_id=response.conversation_id,
    inputs="Resuma toda a conversa em 3 frases.",
)

print(follow_up.outputs[0].content)

Você perguntou sobre as melhores práticas para codificar variáveis categóricas em aprendizado de máquina. Eu listei três técnicas: One-Hot Encoding, Label Encoding e Embedding. Em seguida, você perguntou qual foi a sua primeira pergunta.


<font color="red">``III.`` Listando todas as conversas</font>

* Para gerenciamento interno, você pode ver todas as conversas dos agentes rapidamente.

In [18]:
conversations_list = client.beta.conversations.list()

In [19]:
len(conversations_list)

3

In [20]:
conversations_list[0].id

'conv_01981929efd671908fe477e0bc9cf15c'

* As conversas mais recentes vêm primeiro nesta lista.
* Você pode obter o histórico completo de mensagens com ``get_messages``

In [21]:
conversation = client.beta.conversations.get_messages(conversation_id=conversations_list[0].id)

conversation.messages

[MessageInputEntry(role='user', content='Quais são as melhores práticas para codificar variáveis \n              categóricas em aprendizado de máquina? Liste três.', object='entry', type='message.input', created_at=datetime.datetime(2025, 7, 17, 16, 13, 55, 541356, tzinfo=TzInfo(UTC)), completed_at=None, id='msg_01981929efd5774395e0ea813cc7f090', prefix=False),
 MessageOutputEntry(content='Aqui estão três melhores práticas para codificar variáveis categóricas em aprendizado de máquina:\n\n1. **One-Hot Encoding**:\n   - **Descrição**: Transforma cada categoria em uma coluna binária (0 ou 1). Cada coluna representa uma categoria única.\n   - **Quando usar**: Quando as categorias não têm uma ordem intrínseca (por exemplo, cores, países).\n   - **Exemplo**: Se você tem uma variável "Cor" com categorias "Vermelho", "Azul" e "Verde", o One-Hot Encoding criará três colunas binárias, cada uma representando uma dessas cores.\n\n2. **Label Encoding**:\n   - **Descrição**: Atribui um número intei

In [22]:
len(conversation.messages)

6

* Importante ao construir aplicativos de chat (bate-papo) com histórico de mensagens
* Veja outros [métodos de gerenciamento de conversa](https://docs.mistral.ai/agents/agents_basics/#conversations) na documentação

## <font color="gree">``3.`` Respostas de streaming para facilidade de uso</font>

* Os agentes devem ser amigáveis.
* Transmita respostas sem fazê-los esperar.